In [1]:
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
from langdetect import detect
import numpy as np

In [2]:
with open('all_pk_tweets_all.pickle', "rb") as f:
    tweets = pickle.load(f)

In [3]:
with open("geolocated_users.pickle", "rb") as f:
    geo = pickle.load(f)

In [6]:
#Clean text of self-defined user locations
tweets["user_location"] = tweets["user_location"].str.lower()
tweets["user_location"] = tweets["user_location"].str.replace('[{}]'.format(string.punctuation), '')
tweets["user_location"] = tweets["user_location"].str.strip()

In [7]:
#Identify users who indicate they are in pakistan
tweets["country"] = tweets["user_location"].str.find("pakistan")
conditions = [
    (tweets['country'] >= 0),
    (tweets["country"] < 0)]
choices = ["pakistan", "None"]

tweets["country"] = np.select(conditions, choices, default='null')
tweets["user_location"] = tweets["user_location"].str.replace("pakistan", "")

In [12]:
#Merge geocoordinated user locations with main dataframe
location = tweets[["user_id", "user_location", "country"]]
location = location.groupby(["user_id"]).head(1)
geo = geo[["NAME_3", "index_right"]]
merged = location.merge(geo, how='left', left_on="user_id", right_on="index_right")

In [22]:
#Select users with self-reported locations, clean the text, and substitute geolocated locations where available
users_nan = merged[merged['NAME_3'].isnull()]
users_notnan = merged[merged.NAME_3.notnull()]
users_notnan["NAME_3"] = users_notnan["NAME_3"].str.lower()
users_notnan["NAME_3"] = users_notnan["NAME_3"].str.replace('[{}]'.format(string.punctuation), '')
users_notnan["NAME_3"] = users_notnan["NAME_3"].str.strip()
users_notnan["user_location"] = users_notnan["NAME_3"]
new_users = pd.concat([users_nan, users_notnan], ignore_index = True)
new_users = new_users.drop(columns=["NAME_3", "index_right"])

In [43]:
#Set Pakistan as location for users located in pakistan without more detailed location info
pakistan_locations = location_check[location_check["country"] == "pakistan"]
pakistan_locations["user_location"] = "pakistan"
pakistan_locations = pakistan_locations.drop(columns = ["country"])

In [ ]:
#Combine users with country only data and city level data and determine if city-level data is in english
semi_final_users = pd.concat([location_fine, pakistan_locations], ignore_index = True)

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
    
semi_final_users['english'] = semi_final_users['user_location'].apply(isEnglish)
english_users = semi_final_users[semi_final_users["english"] == True]
non_english_users = semi_final_users[semi_final_users["english"] == False]

In [78]:
#Translated non-english location names
non_english_users["user_location"].value_counts()
pakistan_users = non_english_users[non_english_users["user_location"] == "پاکستان"]
pakistan_users["user_location"] = "pakistan"
lahore_users = non_english_users[non_english_users["user_location"] == "لاہور پاکستان"]
lahore_users["user_location"] = "lahore"
punjab_users = non_english_users[non_english_users["user_location"] == "پنجاب پاکستان"]
punjab_users["user_location"] = "punjab"
karachi_users = non_english_users[non_english_users["user_location"] == "کراچی پاکستان"]
karachi_users["user_location"] = "karachi"
islamabad_users = non_english_users[non_english_users["user_location"] == "اسلام آباد پاکستان"]
islamabad_users["user_location"] = "islamabad"
peshawar_users = non_english_users[non_english_users["user_location"] == "پشاور پاکستان"]
peshawar_users["user_location"] = "peshawar"
rawalpindi_users = non_english_users[non_english_users["user_location"] == "راولپنڈی پاکستان"]
rawalpindi_users["user_location"] = "rawalpindi"
gujranwala_users = non_english_users[non_english_users["user_location"] == "گوجرانوالہ پاکستان"]
gujranwala_users["user_location"] = "gujranwala"
multan_users = non_english_users[non_english_users["user_location"] == "ملتان پاکستان"]
multan_users["user_location"] = "multan"
hyderabad_users = non_english_users[non_english_users["user_location"] == "حیدر آباد پاکستان"]
hyderabad_users["user_location"] = "hyderabad"
sindh_users = non_english_users[non_english_users["user_location"] == "سندھ پاکستان"]
sindh_users["user_location"] = "sindh"
sargodha_users = non_english_users[non_english_users["user_location"] == "سرگودھا پاکستان"]
sargodha_users["user_location"] = "sargodha"
abbottabad_users = non_english_users[non_english_users["user_location"] == "ایبٹ آباد پاکستان"]
abbottabad_users["user_location"] = "abbottabad"
quetta_users = non_english_users[non_english_users["user_location"] == "کوئٹہ پاکستان"]
quetta_users["user_location"] = "quetta"
karachi2_users = non_english_users[non_english_users["user_location"] == "karachi  🇵🇰"]
karachi2_users["user_location"] = "karachi"
lahore2_users = non_english_users[non_english_users["user_location"] == "lahore 🇵🇰"]
lahore2_users["user_location"] = "lahore"

پاکستان                          3361
لاہور پاکستان                    2749
پنجاب پاکستان                    2335
کراچی پاکستان                    1883
اسلام آباد پاکستان               1542
پشاور پاکستان                      26
راولپنڈی پاکستان                   18
گوجرانوالہ پاکستان                 15
ملتان پاکستان                      13
سیالکوٹ پاکستان                    10
حیدر آباد پاکستان                  10
دبي الامارات العربية المتحدة       10
🇵🇰                                 10
سندھ پاکستان                        8
سرگودھا پاکستان                     6
ایبٹ آباد پاکستان                   5
کوئٹہ پاکستان                       5
i̇stanbul türkiye                   5
الامارات العربية المتحدة            5
karachi  🇵🇰                         3
شال                                 2
lahore 🇵🇰                           2
بہاولپور پاکستان                    2
भारत                                2
𝕷𝖆𝖍𝖔𝖗𝖊                              1
مے خانے میں                         1
españa      

In [99]:
#Concatenate all user-location translated users and english language users and clean up final dataframe
translated_users = pd.concat([pakistan_users, lahore_users, karachi_users, punjab_users, islamabad_users, peshawar_users, rawalpindi_users, gujranwala_users, multan_users, hyderabad_users, sindh_users, sargodha_users, abbottabad_users, quetta_users, karachi2_users, lahore2_users], ignore_index = True)
final_users = pd.concat([english_users, translated_users], ignore_index =True)
final_users = final_users.drop(columns = ["english"])
tweets = tweets.drop(columns = ["tweet_id", "user_location", "tweet_longitude", "tweet_latitude", "country"])

In [136]:
#Merge user location data back with tweet data and save
cleaned_users_locations = tweets.merge(final_users, how='left', left_on="user_id", right_on="user_id")
with open("all_tweets_location_cleaned.pickle", "wb") as f:
    pickle.dump(cleaned_users_locations, f)